# Assignment 1


In [1]:
import numpy as np
np.random.seed(13) #TODO Check if this is used for sgd
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Reshape, Lambda
from keras.utils import np_utils
from keras.utils.data_utils import get_file
from keras.preprocessing.text import Tokenizer
from keras.utils.vis_utils import model_to_dot
from keras.preprocessing import sequence
from gensim.models import KeyedVectors
from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors as nn
from matplotlib import pylab
from __future__ import division

C:\Users\Thijs\Anaconda3\envs\Python35\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using Theano backend.
WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
C:\Users\Thijs\Anaconda3\envs\Python35\lib\site-packages\theano\configdefaults.py:560: UserWarning: DeprecationWarning: there is no c++ compiler.This is deprecated and with Theano 0.11 a c++ compiler will be mandatory
  warnings.warn("DeprecationWarning: there is no c++ compiler."
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empt

In [2]:
# DO NOT Modify the lines in this cell
path = 'alice.txt'
corpus = open(path).readlines()[0:700]

corpus = [sentence for sentence in corpus if sentence.count(" ") >= 2]

tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'+"'")
tokenizer.fit_on_texts(corpus)
corpus = tokenizer.texts_to_sequences(corpus)
nb_samples = sum(len(s) for s in corpus)
V = len(tokenizer.word_index) + 1

# Is this something they need to change?
dim = 100
window_size = 2 #use this window size for Skipgram, CBOW, and the model with the additional hidden layer
window_size_corpus = 4 #use this window size for the co-occurrence matrix

## Question 1

### Co-occurrence Matrix
Use the provided code to load the "Alice in Wonderland" text document. 
1. Implement the word-word co-occurrence matrix for “Alice in Wonderland”
2. Normalize the words such that every value lies within a range of 0 and 1
3. Compute the cosine distance between the given words:
    - Alice 
    - Dinah
    - Rabbit
4. List the 5 closest words to 'Alice'. Discuss the results.
5. Discuss what the main drawbacks are of a term-term co-occurence matrix solutions?


In [ ]:
#create co-occurrence matrix
#print(tokenizer)
#print(corpus)
#print(nb_samples)
#print(V)
#print("\n")

def create_co_occurrence(crps, win_size, voc_size):
    # Discuss -1 with Thijs
    co_occurrence_mat = np.zeros((voc_size-1,voc_size-1), int)
    for sentence in crps:
        # Sliding window inside of sentence
        for i, center_word in enumerate(sentence):
            i_min = max(0, i - win_size + 1)
            i_max = min(len(sentence), i + win_size)
            window = sentence[i_min: i_max]
            #print("Window:", window)
            
            # Increment co occurence of words in sliding window
            for j in range(i_min, i_max):
                if i != j:
                    co_word = sentence[j]
                    co_occurrence_mat[center_word-1, co_word-1] += 1
    
    return co_occurrence_mat

co_occurence = create_co_occurrence(corpus, window_size, V)

co_occurence

In [ ]:
#find cosine similarity to Alice, Dinah and Rabbit
from scipy import spatial
print(tokenizer.word_index)

def cosine_sim(word_1, word_2, tknzr, matrix):
    word_1_ind = tknzr.word_index[word_1]
    word_2_ind = tknzr.word_index[word_2]
    
    #print(word_1, word_1_ind)
    #print(word_2, word_2_ind)
    
    word_1_vec = matrix[:,word_1_ind-1]
    word_2_vec = matrix[:,word_2_ind-1]
    
    #print(word_1_vec)
    #print(word_2_vec)
    
    similarity = 1 - spatial.distance.cosine(word_1_vec, word_2_vec)
    #print(similarity)
    
    return similarity
    
    
print('similarity of Alice and Dinah', str(cosine_sim("alice", "dinah", tokenizer, co_occurence)))
print('similarity of Alice and Rabbit', str(cosine_sim("alice", "rabbit", tokenizer, co_occurence)))
print('similarity of Dinah and Rabbit', str(cosine_sim("dinah", "rabbit", tokenizer, co_occurence)))


In [ ]:
#find the closest words to Alice

similarities = [cosine_sim("alice", i, tokenizer, co_occurence) if "alice"!= i else 0.0 for i in tokenizer.word_index]
most_similar_ind = similarities.index(max(similarities)) + 1

print(similarities)
print("Most similar word", most_similar_ind)

Discussion of the drawbacks:

In [ ]:
#Save your all the vector representations of your word embeddings in this way
#Change when necessary the sizes of the vocabulary/embedding dimension

f = open('vectors_co_occurrence.txt',"w")
f.write(" ".join([str(V-1),str(V-1)]))
f.write("\n")

#vectors = your word co-occurrence matrix
vectors = []
for word, i in tokenizer.word_index.items():    
    f.write(word)
    f.write(" ")
    f.write(" ".join(map(str, list(vectors[i,:]))))
    f.write("\n")
f.close()

In [ ]:
#reopen your file as follows

co_occurrence = KeyedVectors.load_word2vec_format('./vectors_co_occurrence.txt', binary=False)

## Question 2

### Word embeddings
Build embeddings with a keras implementation where the embedding vector is of length 50, 150 and 300. Use the Alice in Wonderland text book for training.
1. Using the CBOW model
2. Using Skipgram model
3. Add extra hidden dense layer to CBow and Skipgram implementations. Choose an activation function for that layer and justify your answer.
4. Analyze the four different word embeddings
    - Implement your own function to perform the analogy task with. Do not use existing libraries for this task such as Gensim. Your function should be able to answer whether an anaology as in the example given in the pdf-file is true.
    - Compare the performance on the analogy task between the word embeddings that you have trained in 2.1, 2.2 and 2.3.  
    - Visualize your results and interpret your results
5. Use the word co-occurence matrix from Question 1. Compare the performance on the analogy task with the performance of your trained word embeddings.  
6. Discuss:
    - What are the main advantages of CBOW and Skipgram?
    - What is the advantage of negative sampling?
    - What are the main drawbacks of CBOW and Skipgram?
7. Load pre-trained embeddings on large corpuses (see the pdf file). You only have to consider the word embeddings with an embedding size of 300
    - Compare performance on the analogy task with your own trained embeddings from "Alice in Wonderland". You can limit yourself to the vocabulary of Alice in Wonderland. Visualize the pre-trained word embeddings and compare these with the results of your own trained word embeddings. 


In [7]:
def generate_data_CBOW(corpus, window_size, V):
    maxlen = window_size*2
    all_in = []
    all_out = []
    
    for words in corpus:
        L = len(words)
        for index, word in enumerate(words):
            contexts = []
            labels   = []            
            s = index - window_size
            e = index + window_size + 1
            
            contexts.append([words[i] for i in range(s, e) if 0 <= i < L and i != index])
            labels.append(word)

            x = sequence.pad_sequences(contexts, maxlen=maxlen)
            y = np_utils.to_categorical(labels, V)
            yield (x, y)

In [8]:
x,y = generate_data(corpus, window_size, V)

In [5]:
f = open('data_CBOW.txt' ,'w')

for input,outcome  in zip(x,y):
    input = np.concatenate(input)
    f.write(" ".join(map(str, list(input))))
    f.write(",")
    outcome = np.concatenate(outcome)
    f.write(" ".join(map(str,list(outcome))))
    f.write("\n")
f.close()

In [ ]:
def generate_data_CBOW_from_file():
    f = open('data_CBOW.txt' ,'r')
    for row in f:
        inputs,outputs = row.split(",")
        inputs = np.fromstring(inputs, dtype=int, sep=' ')
        inputs = np.asarray(np.split(inputs, len(inputs)))
        outputs = np.fromstring(outputs, dtype=float, sep=' ')
        outputs = np.asarray(np.split(outputs, len(inputs)))
        yield (inputs,outputs)

In [10]:
#create CBOW model           
cbow = Sequential()
cbow.add(Embedding(input_dim=V, output_dim=dim, input_length=window_size*2))
cbow.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(dim,)))
cbow.add(Dense(V, activation='softmax'))

In [11]:
cbow.compile(loss='categorical_crossentropy', optimizer='adadelta')

In [ ]:
for ite in range(10):
    loss = 0.
    for x, y in generate_data_CBOW(corpus,window_size,V):
        print('start')
        loss += cbow.train_on_batch(x, y)
    print(ite, loss)


start
start
start
start
start
start
start
start


In [ ]:
#prepare data for Skipgram


In [ ]:
#create Skipgram model


In [ ]:
#define loss function for Skipgram


In [ ]:
#train Skipgram model


In [ ]:
#create CBOW model with additional dense layer        
dcbow = Sequential()
dcbow.add(Embedding(input_dim=V, output_dim=dim, input_length=window_size*2))
dcbow.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(dim,)))
dcbow.add(Dense(V, activation='softmax'))

In [ ]:
#define loss function for CBOW + dense


In [ ]:
#train model for CBOW + dense


In [ ]:
#create Skipgram with additional dense layer


In [ ]:
#define loss function for Skipgram + dense


In [ ]:
#train model for Skipgram + dense


In [ ]:
#Implement your own analogy function


Comparison performance:

In [ ]:
#Visualization results trained word embeddings


Interpretation results of the visualization

Compare the results of the trained word embeddings with the word-word co-occurrence matrix

Discussion of the advantages of CBOW and Skipgram, the advantages of negative sampling and drawbacks of CBOW and Skipgram

In [ ]:
#load pretrained word embeddings of word2vec

path_word2vec = "your path /GoogleNews-vectors-negative300.bin"

word2vec = KeyedVectors.load_word2vec_format(path, binary=True)

In [ ]:
#load pretraind word embeddings of Glove

path = "your path /glove.6B/glove.6B.300d_converted.txt"

#convert GloVe into word2vec format
gensim.scripts.glove2word2vec.get_glove_info(path)
gensim.scripts.glove2word2vec.glove2word2vec(path, "glove_converted.txt")

glove = KeyedVectors.load_word2vec_format(path, binary=False)

In [ ]:
#Visualize the pre-trained word embeddings

Comparison performance with your own trained word embeddings